<a href="https://colab.research.google.com/github/vinc-r/projet-carburant/blob/master/explo_donn%C3%A9es_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Études des pompes à essence en France
Étude réalisée par Vincent Rosset, commencée le 12/03/2020

Vincent.Rosset@polytech-lille.net

---




*sources : [https://www.prix-carburants.gouv.fr/rubrique/opendata/](https://www.prix-carburants.gouv.fr/rubrique/opendata/)*

Le site gouvernemental des prix des carburants met à disposition de manière libre et gratuite (Open data) les données relatives aux prix des carburants. Ces données sont référencées sur la plate-forme des données publiques de l’État (www.data.gouv.fr) et sont fournies sous la "[Licence ouverte / Open licence](https://wiki.data.gouv.fr/wiki/Licence_Ouverte_/_Open_Licence)".



## Première feuille : Conversion des xml en csv

La première feuille lit l'ensemble des XML représentant les données étudiés de manière hierarchique et sauvegarde ces données avec le format CSV.
La décompression de ces fichiers est réalisée manuellement.

### Préambule

#### Fonctions utilitaires

Fonctions utilitaires personnalisées

In [0]:
# Timer property decorator
# to get execution time of a function 
def timer(func):
    def f(*args, **kwargs):
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        print('\n\t\t>>>',func.__name__,'execution time:', 
              round(after - before, 4),'secs. <<<')
        return rv
    f.__name__ = func.__name__
    return f

In [0]:
def set_logging_level(log, log_level="INFO"):
    if log_level == "DEBUG":
        log.basicConfig(level=log.DEBUG)
    elif log_level == "INFO":
        log.basicConfig(level=log.INFO)
    elif log_level == "WARNING":
        log.basicConfig(level=log.WARNING)
    elif log_level == "ERROR":
        log.basicConfig(level=log.ERROR)
    elif log_level == "CRITICAL":
        log.basicConfig(level=log.CRITICAL)
    else:
        log.basicConfig(level=log.NOTSET)
    log.info(" > log ready")
    log.debug(" > log level debug")

#### Variables globales

In [0]:
# Start year
START_YEAR = 2007

# End year
END_YEAR = 2018

BASE_URL = "https://storage.googleapis.com/essence-dataset-eda/PrixCarburants_annuel_"

PATH_TO_CSV_FILES = "tmp/"

# choose log level
# Niveau : "DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"
# Attention avant de choisir le niveau DEBUG
#       cela génère de très importantes sorties
LOG_LEVEL = "INFO" 

#### Import packages

In [40]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import xml.sax
from io import BytesIO, StringIO
import requests
from zipfile import ZipFile
from time import time
import os

import logging as log
set_logging_level(log, log_level=LOG_LEVEL)

INFO:root: > log ready


## Lecture des fichiers zippés à partir de Google Storage

Fonctions pour aller lire les données à partir de l'URL renseigné.

In [0]:
def download_datasets(year):
        url = BASE_URL + str(year) + ".zip"
        r = requests.get(url)
        return ZipFile(BytesIO(r.content))

def generator_zip_file():
    for year in range(START_YEAR,END_YEAR+1):
        zip_ref = download_datasets(year)        
        [xml_filename] = zip_ref.namelist()
        yield (zip_ref.open(xml_filename),year)
        zip_ref.close()

Création d'une classe à laquelle on apprend comment le jeu de données doit être lu.

In [0]:
class StreamHandler(xml.sax.handler.ContentHandler):

    def __init__(self, year, pdv_csv, prix_csv, 
                 ouverture_csv, services_csv, 
                 fermeture_csv, rupture_csv):
        # csv d'écriture
        self.pdv_csv = pdv_csv
        self.prix_csv = prix_csv
        self.ouverture_csv = ouverture_csv
        self.services_csv = services_csv
        self.fermeture_csv = fermeture_csv
        self.rupture_csv = rupture_csv
        # dictionnaire de stockage des info pdv
        self.found = {
            "pdv" : {}, "adresse" : "", "ville" : "", "ouverture" : {},
            "jour" : {}, "horaire" : {}, "service" : "",
            "prix" : {}, "fermeture" : {}, "rupture" : {}
        }
        # tag actuel ouvert
        self.actual_tag = None
        # année en cours
        self.year = year
        log.info(" > StreamHandler init > DONE")

    def startElement(self, name, attrs):
        """
        Appelé à chaque tag ouvrant, par exemple <coucou>. 
        Les attributs du tag sont disponibles sous forme de dictionnaire.
        Ils sont stockés ensuite selon le nom du tag.
        """
        # si le tag est reconnu => traitement
        if name in self.found.keys():
            # si le tag attend des attribus => stockage des attribus
            if type(self.found[name]) == dict:
                log.debug(" > start tag > " + name + \
                          " : " + str(dict(attrs.items())))
                self.found[name] = dict(attrs.items())
            else:
                log.debug(" > start tag > " + name)
            # MAJ tag actuel ouvert
            self.actual_tag = name

    def characters(self, content):
        """
        Appelé à chaque texte entre les tags, par exemple pour 
        <coucou>ICI</coucou>AUSSI<tralala/> la méthode sera appelé 
        pour ICI puis pour AUSSI
        """
        # concatenation de tous les eléments de text
        # pour éviter les séparations indésirées des caratères spéciaux
        if self.actual_tag is not None and \
        type(self.found[self.actual_tag]) == str:
            self.found[self.actual_tag] += content
            log.debug(" > content : " + self.found[self.actual_tag])
        
    def endElement(self, name):
        """
        Appelé à chaque tag fermant, par exemple </coucou>
        """
        # écriture des jours de services
        if name == "service":
            d = {"id_pdv" : self.found["pdv"]["id"],
                 "year" : self.year,
                 "service" : self.found[name]}
            log.debug(" > write service row : " + str(d))
            self.services_csv.writerow(d)
            self.found[name] = ""

        # écriture des jours de fermeture
        # format des jours d'ouverture pour années 2007 à 2017 inclu
        elif name == "ouverture" and len(self.found[name]) > 0:
            for day in ["Lundi", "Mardi", "Mercredi", 
                        "Jeudi", "Vendredi", "Samedi", "Dimanche"]:
                if day not in self.found[name]:
                    d = {"id_pdv" : self.found["pdv"]["id"],
                         "year" : self.year,
                         "debut" : self.found[name]["debut"],
                         "fin" : self.found[name]["fin"],
                         "jour" : day}
                    log.debug(" > write day on row : " + str(d))
                    self.ouverture_csv.writerow(d)

        # nouveau format des ouverture à partir de 2018
        # jour off non pris en compte car sans taf horaire
        elif name == "horaire":
            d = {"id_pdv" : self.found["pdv"]["id"],
                "year" : self.year,
                "debut" : self.found[name]["ouverture"],
                "fin" : self.found[name]["fermeture"],
                "jour" : self.found["jour"]["nom"]}
            log.debug(" > write day on row : " + str(d))
            self.ouverture_csv.writerow(d)
            self.found[name] = {}

        elif name == "jour":
            # Uniquement RAZ du dictionnaire
            # info déjà écrite avec horaire
            # non RAZ avec horaire car il peut y avoir plusieurs horaire / jour
            self.found[name] = {}

        # type d'écriture identique sur les fichiers suivant
        # => simplification code par boucle
        # écriture des prix, fermeture, rupture
        for f, n in ([self.prix_csv, "prix"],
                     [self.fermeture_csv, "fermeture"],
                     [self.rupture_csv, "rupture"]):
            if name == n and len(self.found[name]) > 0:
                self.found[n].update({
                    "id_pdv" : self.found["pdv"]["id"],
                    "year" : self.year})
                log.debug(" > write " + n + " row : " + str(self.found[n]))
                f.writerow(self.found[n])
                self.found[n] = {}

        # si c'est une balise pdv qui est fermée,
        # RAZ dictionnaire et écriture pdv
        if name == "pdv":
            log.debug(" > write pdv row : " + str(self.found["ouverture"]))
            self.found["pdv"].update({
                "year" : self.year,
                "adresse": self.found["adresse"],
                "ville" : self.found["ville"]
            })
            log.debug(" > write pdv row : " + str(self.found["pdv"]))
            self.pdv_csv.writerow(self.found["pdv"])
            # RAZ du dictionnaire de stockage
            self.found = {
                "pdv" : {}, "adresse" : "", "ville" : "", "ouverture" : {},
                "jour" : {}, "horaire" : {}, "service" : "",
                "prix" : {}, "fermeture" : {}, "rupture" : {}
            }
            log.debug(" > RAZ found elements")
        # RAZ du tag
        self.actual_tag = None

Fonction d'ouvertue d'un fichier csv pour écriture.

In [0]:
def open_csv(csv_name, fnames):
    f = open(csv_name, 'w', encoding='utf-8')
    file_csv = csv.DictWriter(f, fieldnames=fnames)
    file_csv.writeheader()
    log.info(" > File : " + f.name + " > OPEN")
    return file_csv, f

Fonction principale :
1.   Ouverture des cvs
2.   création du parser et du handler
3.   parcours des xml sur toutes les années
4.   écriture ligne par ligne dans les csv
5.   fermeture des csv



La propriété *@timer* permet d'afficher la durée d'exécution de la fonction.

In [0]:
@timer
def export_xml_to_csv():

    # création dossier tmp où seront stockés les csv
    if not os.path.exists(PATH_TO_CSV_FILES):
        os.makedirs(PATH_TO_CSV_FILES)
        log.info(" > Folder : " + PATH_TO_CSV_FILES + " > CREATED")

    # ouverture des csv
    pdv_csv, pdv_f = open_csv(PATH_TO_CSV_FILES + "pdv.csv", 
                            ['id', 'year', 'latitude', 'longitude', 'cp', 
                            'pop', 'adresse', 'ville'])

    prix_csv, prix_f = open_csv(PATH_TO_CSV_FILES + "prix.csv", 
                                ['id_pdv', 'year', 'nom', 
                                 'id', 'maj', 'valeur'])

    ouverture_csv, ouverture_f = open_csv(PATH_TO_CSV_FILES + "ouverture.csv", 
                                          ['id_pdv', 'year', 
                                           'jour', 'debut', 'fin'])

    services_csv, services_f = open_csv(PATH_TO_CSV_FILES + "services.csv", 
                                        ['id_pdv', 'year', 'service'])

    fermeture_csv, fermeture_f = open_csv(PATH_TO_CSV_FILES + "fermeture.csv",
                                        ['id_pdv', 'year', 
                                        'type', "debut", "fin"])

    rupture_csv, rupture_f = open_csv(PATH_TO_CSV_FILES + "rupture.csv",
                                    ['id_pdv', 'year', 
                                    'id', "nom", "debut", "fin"])

    parser = xml.sax.make_parser()
    handler = StreamHandler(START_YEAR, pdv_csv, prix_csv, ouverture_csv,
                            services_csv, fermeture_csv, rupture_csv)
    log.info(" > Handler > READY")
    parser.setContentHandler(handler)
    log.info(" > Parser > READY")

    for file,year in generator_zip_file():
        log.info(" > Year : " + str(year) + " > file found : " + str(file))
        # MAJ year sur handler
        handler.year = year
        # lecture xml
        my_xml = StringIO(file.read().decode("utf-8"))
        # perser xml
        parser.parse(my_xml)
        # fermer fichier de lecture
        file.close()
        log.info(" > Year : " + str(year) + " > DONE")

    # fermeture de tous les fichiers d'écriture
    for f in [pdv_f, prix_f, ouverture_f, services_f, fermeture_f, rupture_f]:
        f.close()
        log.info(" > File : " + f.name + " > CLOSE")

Appel fonction principale

In [45]:
export_xml_to_csv()

INFO:root: > File : tmp/pdv.csv > OPEN
INFO:root: > File : tmp/prix.csv > OPEN
INFO:root: > File : tmp/ouverture.csv > OPEN
INFO:root: > File : tmp/services.csv > OPEN
INFO:root: > File : tmp/fermeture.csv > OPEN
INFO:root: > File : tmp/rupture.csv > OPEN
INFO:root: > StreamHandler init done
INFO:root: > Handler > READY
INFO:root: > Parser > READY
INFO:root: > Year : 2007 > file found : <zipfile.ZipExtFile name='PrixCarburants_annuel_2007.xml' mode='r' compress_type=deflate>
INFO:root: > Year : 2007 > DONE
INFO:root: > Year : 2008 > file found : <zipfile.ZipExtFile name='PrixCarburants_annuel_2008.xml' mode='r' compress_type=deflate>
INFO:root: > Year : 2008 > DONE
INFO:root: > Year : 2009 > file found : <zipfile.ZipExtFile name='PrixCarburants_annuel_2009.xml' mode='r' compress_type=deflate>
INFO:root: > Year : 2009 > DONE
INFO:root: > Year : 2010 > file found : <zipfile.ZipExtFile name='PrixCarburants_annuel_2010.xml' mode='r' compress_type=deflate>
INFO:root: > Year : 2010 > DONE
IN


		>>> export_xml_to_csv execution time: 545.7057 secs. <<<




---


FIN FEUILLE CONVERTION XML EN CSV

---

